In [1]:
import tensorflow

In [2]:
import keras

Using TensorFlow backend.


In [3]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [9]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18270953556484285076
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11285974221
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8600971401502338253
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [4]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [10]:
img_width, img_height = 128, 128
train_data_dir = "/data/train_binary"
validation_data_dir = "/data/train_binary"
nb_train_samples = 2942
nb_validation_samples = 326 
batch_size = 16
epochs = 50

In [11]:
nr_categories = 2
batch_size = 16
epochs = 50

In [12]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#from keras.models import load_model
#model = load_model("/data/vgg16_1.h5")

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [14]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
# for layer in model.layers[:12]:
#     print(layer.name)
#     layer.trainable = False

In [15]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
#x = Dense(1024, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [16]:
model_final = Model(input = model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [17]:
# Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in model_final.layers[:12]:
    print(layer.name)
    layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool


In [18]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [19]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


In [20]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.1)

In [21]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [22]:
img_width, img_height = 128, 128
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 2942 images belonging to 2 classes.


In [23]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 326 images belonging to 2 classes.


In [24]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_binary_cat200_cat300_128_p2.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [25]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [26]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
184/183 [==============================] - 88s 477ms/step - loss: 0.1496 - acc: 0.9351 - val_loss: 0.0972 - val_acc: 0.9724

Epoch 00001: val_acc improved from -inf to 0.97239, saving model to vgg16_binary_cat200_cat300_128_p2.h5
Epoch 2/50
184/183 [==============================] - 78s 422ms/step - loss: 0.0264 - acc: 0.9912 - val_loss: 0.0903 - val_acc: 0.9816

Epoch 00002: val_acc improved from 0.97239 to 0.98160, saving model to vgg16_binary_cat200_cat300_128_p2.h5
Epoch 3/50
184/183 [==============================] - 77s 420ms/step - loss: 0.0175 - acc: 0.9932 - val_loss: 0.1035 - val_acc: 0.9816

Epoch 00003: val_acc did not improve from 0.98160
Epoch 4/50
184/183 [==============================] - 77s 420ms/step - loss: 0.0081 - acc: 0.9973 - val_loss: 0.0962 - val_acc: 0.9877

Epoch 00004: val_acc improved from 0.98160 to 0.98773, saving model to vgg16_binary_cat200_cat300_128_p2.h5
Epoch 5/50
184/183 [==============================] - 78s 421ms/step - loss: 0.0060 -

In [ ]:
!ls

In [ ]:
batch_size = 16

generator = train__val_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=int(batch_size/10),
        class_mode=None) #,  # this means our generator will only yield batches of data, no labels
#) #shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(generator, 60000)
# save the output as a Numpy array
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

generator = train__val_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=int(batch_size/20),
        class_mode=None)
#) #shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, 8000)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

In [ ]:
train_data = np.load(open('bottleneck_features_train.npy'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nr_categories, activation='softmax')) 
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

In [ ]:
print(1)